In [1]:
from  google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [8]:
# Leer el archivo CSV
archivo_csv = '/content/drive/MyDrive/MIT micromaster/Grupo 3 - DATA/df_result_3_tomodel_noh_enaho.csv'#'/content/drive/MyDrive/Grupo 3 - DATA/df_result_2_tomodel.csv'
datos = pd.read_csv(archivo_csv)

In [9]:
# Add Mes-num
meses = ['ENERO', 'FEBRERO','MARZO','ABRIL','MAYO','JUNIO','JULIO', 'AGOSTO','SETIEMBRE','OCTUBRE','NOVIEMBRE', 'DICIEMBRE' ]
diccionario = {mes : i+1 for i, mes in enumerate(meses)}
datos["MES_NUM"] = datos["MES"].apply(lambda x: diccionario.get(x,None))

In [10]:
datos.sort_values(by=['VACUNA','PROVINCIA','ANIO','MES_NUM'], inplace=True)
datos.rename(columns={"CANTIDAD DE PERSONAL": "CANTIDAD_DE_PERSONAL"}, inplace = True)
datos["ANIO"] = datos["ANIO"].astype(str)

In [11]:
# Shifting
df_list=[]
for item,df_group in datos.groupby(['PROVINCIA','VACUNA']):
  df_group.set_index(['VACUNA', 'ANIO', 'MES', 'PROVINCIA', 'RANGO_EDAD',
       'CANTIDAD_DE_PERSONAL', 'POBLACION', 'PRESUPUESTO',
       'UBIGEO', 'ALFABETISMO', 'AGUA_POTABLE', 'DEMORA_CITA_horas',
       'DEMORA_TRASLADO_horas', 'MES_NUM'], inplace = True)
  df_group_shifted = df_group.shift(2)
  df_group_shifted.reset_index(inplace=True)
  df_list.append(df_group_shifted)
df_shifted_total = pd.concat(df_list)
datos = df_shifted_total.dropna(subset=['CANTIDAD'])

In [13]:
# Separar las características (X) y la variable objetivo (y)
X = datos.drop('CANTIDAD', axis=1)
y = datos['CANTIDAD']
# Split de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Obtener las columnas numéricas
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Obtener las columnas categóricas
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

# Definir transformadores para las columnas categóricas y numéricas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combinar los transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Inicialización del modelo de regresión lineal
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

In [14]:
# Inicialización del modelo de regresión lineal
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])
# Entrenamiento del modelo
model.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluación del modelo en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")

# Tuning de hiperparámetros utilizando k-fold cross-validation
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validation Mean Squared Error: {-scores.mean()}")

# Baselines
mean_baseline_mse = mean_squared_error(y_test, np.full_like(y_test, y_train.mean()))
max_baseline_mse = mean_squared_error(y_test, np.full_like(y_test, y_train.max()))
print(f"Mean Baseline MSE: {mean_baseline_mse}")
print(f"Max Baseline MSE: {max_baseline_mse}")

Mean Squared Error on Test Set: 9226.95209186551
Cross-Validation Mean Squared Error: 9882.744500946097
Mean Baseline MSE: 26708.82107498612
Max Baseline MSE: 1754131.3541666667


In [16]:
datos = datos[datos['PROVINCIA']!='PURUS']

In [17]:
# Separar las características (X) y la variable objetivo (y)
X = datos.drop('CANTIDAD', axis=1)
y = datos['CANTIDAD']
# Split de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Obtener las columnas numéricas
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Obtener las columnas categóricas
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()

# Definir transformadores para las columnas categóricas y numéricas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combinar los transformadores en un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Inicialización del modelo de regresión lineal
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

In [18]:
# Inicialización del modelo de regresión lineal
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])
# Entrenamiento del modelo
model.fit(X_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluación del modelo en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")

# Tuning de hiperparámetros utilizando k-fold cross-validation
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validation Mean Squared Error: {-scores.mean()}")

# Baselines
mean_baseline_mse = mean_squared_error(y_test, np.full_like(y_test, y_train.mean()))
max_baseline_mse = mean_squared_error(y_test, np.full_like(y_test, y_train.max()))
print(f"Mean Baseline MSE: {mean_baseline_mse}")
print(f"Max Baseline MSE: {max_baseline_mse}")

Mean Squared Error on Test Set: 12169.866001411381
Cross-Validation Mean Squared Error: 11755.380361189802
Mean Baseline MSE: 30020.693627391192
Max Baseline MSE: 1689462.9132075473


##Borrador


In [ ]:
# Separar las características (X) y la variable objetivo (y)
X = datos.drop('CANTIDAD', axis=1)
y = datos['CANTIDAD']

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% evaluación)
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definir el modelo de regresión lineal
modelo = LinearRegression()
X_train = pd.get_dummies(X_train)
# Definir los parámetros para la búsqueda de hiperparámetros
parametros = {'fit_intercept': [True, False]}

# Realizar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(modelo, parametros, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros
mejores_parametros = grid_search.best_params_

In [ ]:
mejores_parametros

{'fit_intercept': True}

In [ ]:
# Entrenar el modelo con los mejores parámetros
modelo_final = LinearRegression(**mejores_parametros)
modelo_final.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Hacer predicciones en el conjunto de evaluación
X_eval = pd.get_dummies(X_eval)
predicciones_eval = modelo_final.predict(X_eval)

# Calcular el error cuadrático medio en el conjunto de evaluación
mse_eval = mean_squared_error(y_eval, predicciones_eval)
mape = mean_absolute_percentage_error(y_eval, predicciones_eval)
r2 = r2_score(y_eval, predicciones_eval)
print(f"Error cuadrático medio en el conjunto de evaluación: {mse_eval}")
print(f"mape en el conjunto de evaluación: {mape}")
print(f"r2 en el conjunto de evaluación: {r2}")

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros:")
print(mejores_parametros)

Error cuadrático medio en el conjunto de evaluación: 6791.186836518047
mape en el conjunto de evaluación: 3528297797239309.0
r2 en el conjunto de evaluación: 0.6878493650625319
Mejores parámetros:
{'fit_intercept': True}


In [ ]:
# Hacer predicciones en el conjunto de evaluación
X_eval = pd.get_dummies(X_eval)
predicciones_eval = modelo_final.predict(X_eval)

# Calcular el error cuadrático medio en el conjunto de evaluación
mse_eval = mean_squared_error(np.exp(y_eval)-1, np.exp(predicciones_eval)-1)
mape = mean_absolute_percentage_error(np.exp(y_eval)-1, np.exp(predicciones_eval)-1)
r2 = r2_score(y_eval, predicciones_eval)
print(f"Error cuadrático medio en el conjunto de evaluación: {mse_eval}")
print(f"mape en el conjunto de evaluación: {mape}")
print(f"r2 en el conjunto de evaluación: {r2}")

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros:")
print(mejores_parametros)

Error cuadrático medio en el conjunto de evaluación: 4901.033101479071
mape en el conjunto de evaluación: 393950084852602.2
r2 en el conjunto de evaluación: 0.8751193211225341
Mejores parámetros:
{'fit_intercept': True}


In [ ]:
dic = {"y_eval": y_eval, "predicciones_eval": predicciones_eval}
df_eval = pd.DataFrame(dic)
df_eval["dif_abs"]=np.abs((np.exp(df_eval["y_eval"])-1 - np.exp(df_eval["predicciones_eval"])+1) / (np.exp(df_eval["y_eval"])-1+0.001))
df_eval.sort_values(by=["dif_abs"],ascending=False)

,y_eval,predicciones_eval,dif_abs
1626,0.000000,3.144875,22216.763276
4275,0.000000,3.054977,20220.706294
1147,0.000000,2.793861,15344.008689
5768,0.000000,2.735580,14418.690531
5767,0.000000,2.706581,13977.979898
...,...,...,...
144,2.079442,2.077797,0.001878
8372,4.727388,4.728912,0.001539
3582,1.098612,1.097694,0.001376
3254,3.332205,3.330935,0.001316


In [ ]:
np.mean(df_eval["dif_abs"])

262.05643288540193